In [62]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

BASE_URL = "https://www.sentinelassam.com/"
country = 'bangladesh'
initial_url = f'{BASE_URL}search?q={country}'

#LOADING USING SELENIUM AS 'READ MORE' BUTTON NEEDS TO BE PRESSED, AND USING BEAUTIFUL SOUP FOR SCRAPING

driver = webdriver.Chrome()
driver.get(initial_url)

time.sleep(10)

button = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[1]/div[2]/div/div/div[2]/div')

ActionChains(driver)\
.click(button)\
.perform()

time.sleep(10)

soup = BeautifulSoup(driver.page_source, 'html.parser')

link_divs = soup.find_all('div',{'data-test-id':'headline'})

all_news_links = []

if len(link_divs) > 0:
    
    for each_link in link_divs:
        
        all_news_links.append(each_link.find('a').get('href'))
        
driver.quit()

#SCRAPING ALL NEWS LINKS

counter = 0
data_list = []

for url in all_news_links:

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('h1',{'data-testid':'story-headline'})
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #CONTENT SUMMARY

        content_summary_tag = soup.find('div',{'data-test-id':'subheadline'})
        content_summary = content_summary_tag.text if content_summary_tag else 'Content Summary not found'
        summary_translation = 'None'

        #Author

        author_link = soup.find('a', {'aria-label':'author-name'})
        author = author_link.text if author_link else 'Author not found'

        #Date

        date_info = soup.find('time', class_ = 'arr__timeago')

        date_data = date_info.text if date_info else 'Date data not found'

        if date_data != 'Date data not found':

            source_localtime = datetime.strptime(date_data, '%d %b %Y, %H:%M %p')
            bangladesh_localtime = source_localtime + timedelta(minutes=30)


        #CONTENT

        content = []

        main_content_div = soup.find('div', class_ = 'arr--story-page-card-wrapper')

        if main_content_div:

            all_paras = main_content_div.find_all('p')

            for each_para in all_paras:

                content.append(each_para.text)

            full_content = ''.join(content)

            full_content = full_content.replace('\xa0','')

            also_read_template = full_content.split('Also Read:')

            if len(also_read_template) > 1:

                full_content = also_read_template[0]

            else:
                pass

        else:

            full_content = 'Content not found'
            
        content_translation = 'None'    
        
        data_dict = {
        "url": url,
        "title": title,
        "content": full_content,
        "content_summary": content_summary,
        "title_translation":title_translation,
        "content_translation":content_translation,
        "summary translation":summary_translation,
        "author": author,
        "country": country,
        'source_localtime': source_localtime,
        'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content not Found" and title != 'Title not found' and date_data != 'Date data not found'):

                if data_dict not in data_list:
                    # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
            

df = pd.DataFrame(data_list)
df.head()


csv_filename = f"{country}_The_Sentinel_Assam.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)

Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.sentinelassam.com/north-east-india...,Assam: Cattle Smuggler Killed in BSF Firing Al...,SOUTH SALMARA: A tragic eve­nt took place on t...,"Sahinur Islam, a 26-year-old cattle smuggler, ...",None,None,None,Sentinel Digital Desk,bangladesh,2024-02-22 01:58:00,2024-02-22 02:28:00
1,https://www.sentinelassam.com/more-news/intern...,Bangladesh Islami Front calls for closer Delhi...,New Delhi: Bangladesh’s far right political pa...,Bangladesh’s far right political party Banglad...,None,None,None,Sentinel Digital Desk,bangladesh,2024-02-21 05:31:00,2024-02-21 06:01:00
2,https://www.sentinelassam.com/sports-news/bang...,Bangladesh Fast bowler Mustafizur Rahman taken...,NEW DELHI: Bangladesh Fast bowler Mustafizur R...,Bangladesh Fast bowler Mustafizur Rahman has b...,None,None,None,Sentinel Digital Desk,bangladesh,2024-02-19 05:33:00,2024-02-19 06:03:00
3,https://www.sentinelassam.com/north-east-india...,Assam: Gold biscuits worth Rs 2.3 crore seized...,A CORRESPONDENTDHUBRI: The Golokganj police of...,"The Golokganj police of Dhubri, acting on a ti...",None,None,None,Sentinel Digital Desk,bangladesh,2024-02-14 04:46:00,2024-02-14 05:16:00
4,https://www.sentinelassam.com/more-news/editor...,Stable Bangladesh,President Droupadi Murmu has made a very signi...,President Droupadi Murmu has made a very signi...,None,None,None,Sentinel Digital Desk,bangladesh,2024-02-11 04:53:00,2024-02-11 05:23:00
